# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [11]:
import numpy as np
import pandas as pd
import os
import glob 
import os

from datetime import datetime
from Train_Model import clean_data
from sklearn.model_selection import train_test_split

In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
experiment_name = 'Hyp-tun-bankrupt'

experiment=Experiment(ws, experiment_name)


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: ops_analytics-workspace
Azure region: westeurope
Subscription id: 73f88e6b-3a35-4612-b550-555157e7059f
Resource group: ops_analytics-rg-gb-dev


# Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [13]:
#from azureml.core import Dataset
#from azureml.core import Workspace

#data = pd.read_csv('payer_level_features.csv')
#x, y = clean_data(data)

#all_data = pd.concat([x,y], axis = 1)

#Train, Test = train_test_split(
       #all_data , test_size=0.33, random_state=0)

#ws = Workspace.from_config()


#Train.to_csv("Data/Train.csv", index = False)

#path = "Data/Train.csv"

#data_store = ws.get_default_datastore()

#try:
 #   train_dataset = Dataset.get_by_name(ws, name="Train")
#except: 
 #   data_store.upload(src_dir = "./Data", target_path = './Data', overwrite = True,  show_progress = True) 
        
# Create TabularDataset & register in workspace
#poc_dataset = Dataset.Tabular.from_delimited_files([(data_store, path)])
#poc_dataset = poc_dataset.register(
 #       ws, name="poc_dataset", create_new_version=True,
  #      description="Dataset for POC Bankrupt prediction")  

Uploading an estimated of 4 files
Uploading ./Data/.amlignore
Uploaded ./Data/.amlignore, 1 files out of an estimated total of 4
Uploading ./Data/.amlignore.amltmp
Uploaded ./Data/.amlignore.amltmp, 2 files out of an estimated total of 4
Uploading ./Data/Train.csv
Uploaded ./Data/Train.csv, 3 files out of an estimated total of 4
Uploading ./Data/train_data.csv
Uploaded ./Data/train_data.csv, 4 files out of an estimated total of 4
Uploaded 4 files


## Choose the Compute Cluster 

In [15]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException 

# Choose a name for your CPU cluster
amlcompute_cluster_name = "Demo-Cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print("Creating a new compute cluster...")
    compute_config =  AmlCompute.provisioning_configuration(vm_size='STANDARD_D2S_V3', max_nodes=4, min_nodes=0, vm_priority='lowpriority') 
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
    
#aml_compute.wait_for_completion(show_output=True)
aml_compute.wait_for_completion(show_output=True, min_node_count = None, timeout_in_minutes = 60)
#compute_target.wait_for_completion(show_output=True, min_node_count = None, timeout_in_minutes = 60)

# use get_status() to get a detailed status for the current cluster. 
print(aml_compute.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-11-21T04:13:32.468000+00:00', 'errors': None, 'creationTime': '2021-09-17T05:18:08.686648+00:00', 'modifiedTime': '2021-09-17T05:18:24.335748+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2S_V3'}


In [19]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# to install required packages
env = Environment('tutorial-env')
cd = CondaDependencies.create(pip_packages=['azureml-dataset-runtime[pandas,fuse]', 'azureml-defaults'], conda_packages = ['scikit-learn==0.22.1'])

env.python.conda_dependencies = cd

# Register environment to re-use later
env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210714.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "tutorial-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
 

In [60]:
from azureml.core import ScriptRunConfig

args = ['--n_estimators', 100, '--max_depth', 7  , '--max_features' , 0.25 , '--n_jobs' , -1 ]

src = ScriptRunConfig(source_directory='.',
                      script='Train_Model.py', 
                      arguments=args,
                      compute_target=aml_compute,
                      environment=env)

In [61]:
run = experiment.submit(config=src)

Submitting /mnt/batch/tasks/shared/LS_root/mounts/clusters/trial/code/Users/narendhrakumar.murugan/starter_file directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


In [62]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [68]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform , choice
from azureml.train.estimator import Estimator
from azureml.train.hyperdrive import BayesianParameterSampling


#early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = BayesianParameterSampling({
                    'learning_rate': uniform(0.05, 0.1),
                    'batch_size': choice(16, 32, 64, 128)  })


hyperdrive_config = HyperDriveConfig(
                    hyperparameter_sampling=param_sampling,
                    primary_metric_name='accuracy',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_concurrent_runs = 4,    
                    max_total_runs=40,
                    run_config = src)

In [ ]:
#TODO: Submit your experiment

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [69]:
hyperdrive_run = experiment.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
# wait for completion
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_c219e294-534a-49d8-847c-5655974e6b91
Web View: https://ml.azure.com/runs/HD_c219e294-534a-49d8-847c-5655974e6b91?wsid=/subscriptions/73f88e6b-3a35-4612-b550-555157e7059f/resourcegroups/ops_analytics-rg-gb-dev/workspaces/ops_analytics-workspace&tid=cef04b19-7776-4a94-b89b-375c77a8f936

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-11-22T17:31:10.589933][API][INFO]Experiment created<END>\n""<START>[2021-11-22T17:31:11.433772][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-11-22T17:31:11.644347][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_c219e294-534a-49d8-847c-5655974e6b91
Web View: https://ml.azure.com/runs/HD_c219e294-534a-49d8-847c-5655974e6b91?wsid=/subscriptions/73f88e6b-3a35-4612-b550-555157e7059f/resourcegroups/ops_analytics-rg-gb-dev/workspaces/ops_analytics-workspace&tid=cef04b19-7776-4a94-b89b-375c77a8f936



{'runId': 'HD_c219e294-534a-49d8-847c-5655974e6b91',
 'target': 'Demo-Cluster',
 'status': 'Canceled',
 'startTimeUtc': '2021-11-22T17:31:10.204863Z',
 'endTimeUtc': '2021-11-22T17:39:14.098883Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'properties': {'primary_metric_config': '{"name": "accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'a8b128ab-09f4-43f5-828a-8039143f5124',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.33.0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://opsanalyticswo3045522382.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_c219e294

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

